In [2]:
!pip install textblob

from textblob import TextBlob

import pandas as pd

import requests

import json

In [3]:
#The 2020 MLB Draft Class

mlb_json_id_url = 'http://m.mlb.com/gen/players/prospects/2020/playerProspects.json'
response = requests.get(mlb_json_id_url)
response.status_code
all_json = response.json()
draft_prospect_list_2020 = all_json['prospect_players']['draft']
draft_player_ids_2020 = []
draft_player_ranks_2020 = []
draft_player_first_name_2020 = []
draft_player_last_name_2020 = []
draft_player_position_2020 = []
for i in draft_prospect_list_2020:
    player_id_2020 = str(i['player_id'])
    draft_player_ids_2020.append(player_id_2020)                      
    rank_2020 = i["rank"]
    draft_player_ranks_2020.append(rank_2020)
    first_name_2020 = i["player_first_name"]
    draft_player_first_name_2020.append(first_name_2020)
    last_name_2020 = i["player_last_name"]
    draft_player_last_name_2020.append(last_name_2020)
    position_2020 = i["position"]
    draft_player_position_2020.append(position_2020)
    player_id_data_2020 = {'First Name' : draft_player_first_name_2020, 'Last Name' : draft_player_last_name_2020, 'Player ID' : draft_player_ids_2020, 'Position' : draft_player_position_2020, 'Player Rank' : draft_player_ranks_2020}

draft_player_id_dataframe_2020 = pd.DataFrame(player_id_data_2020)


###Part 3: Scraping every Top100 report into dataframes
pipeline_url = "http://m.mlb.com/gen/players/prospects/2020/"
draft_prospect_url_list_2020 = []
link_status_2020 = []
#making links
for i in draft_player_ids_2020:
    prospect_url = pipeline_url + i + '.json'
    draft_prospect_url_list_2020.append(prospect_url)

#testing links
for i in draft_prospect_url_list_2020:    
    response = requests.get(i)
    if response.status_code == 200:
        link = 'Link is working'
    else:
        link = 'Link does not work'
    link_status_2020.append(link)
    
#making initial list of player scout-reports/cards
draft_card_list_2020 = []
for i in draft_prospect_url_list_2020:    
    response = requests.get(i)
    player_card_framed_2020 = response.json()
    player_card_2020 = player_card_framed_2020['prospect_player']
    player_card_2020
    player_report_and_video_2020 = player_card_2020['content']
    player_report_2020 = player_report_and_video_2020['default']
    draft_card_list_2020.append(player_report_2020)



draft_report_trim_1_list_2020 = []
for i in draft_card_list_2020:
    trim = i.rfind('Overall: ') + 10
    report_trim_1 = i[trim:]
    draft_report_trim_1_list_2020.append(report_trim_1)

draft_report_cleaner_list_2020 = []
for i in draft_report_trim_1_list_2020:
    if i.find('</a>') == -1:
        start = i.find('n<p>') + 3
    else:
        start = i.find('</a>') + 8
    trim_a = i[start:]
    draft_report_cleaner_list_2020.append(trim_a)

draft_reports_2020 = []
for i in draft_report_cleaner_list_2020:
    without_p = i.replace('<p>', '')
    without_p2 = without_p.replace('p>', '')
    without_n = without_p2.replace('\n', '')
    lowered = without_n.lower()
    draft_reports_2020.append(lowered)

draft_player_id_dataframe_2020['Written Report'] = draft_reports_2020
draft_top_one_hundred_data_2020 = draft_player_id_dataframe_2020


full_draft_2020 = []

for i in draft_reports_2020:
    
    draft_text_2020 = TextBlob(i)
   
    draft_score_2020 = draft_text_2020.sentiment
    
    full_draft_2020.append(draft_score_2020)


draft_top_one_hundred_data_2020['Sentiment Score'] = full_draft_2020

sorted_draft_2020 = draft_top_one_hundred_data_2020.sort_values(by=['Sentiment Score'],ascending = False)
sorted_draft_2020 = sorted_draft_2020.drop(['Written Report', 'Player ID'], axis = 1)
sorted_draft_2020

,First Name,Last Name,Position,Player Rank,Sentiment Score
38,Alex,Santos,RHP,39,"(0.4258663558663558, 0.5614491064491065)"
13,Reid,Detmers,LHP,14,"(0.2946153846153846, 0.3765384615384616)"
89,Timmy,Manning,LHP,90,"(0.28607638888888887, 0.4532638888888889)"
81,Carter,Baumler,RHP,82,"(0.2768421052631579, 0.44333333333333325)"
1,Spencer,Torkelson,1B,2,"(0.27666666666666667, 0.41000000000000003)"
18,Carmen,Mlodzinski,RHP,19,"(0.27249999999999996, 0.32611111111111113)"
97,Ryan,Bruno,LHP,98,"(0.2648888888888889, 0.4328888888888889)"
59,Nick,Frasso,RHP,60,"(0.2605483405483406, 0.5197988122988123)"
30,Carson,Montgomery,RHP,31,"(0.2512247474747475, 0.4339500777000777)"
98,Jackson,Miller,C,99,"(0.24452380952380953, 0.47397108843537417)"


In [4]:
#The 2019 MLB Draft Class

mlb_json_id_url = 'http://m.mlb.com/gen/players/prospects/2019/playerProspects.json'
response = requests.get(mlb_json_id_url)
response.status_code
all_json = response.json()
draft_prospect_list_2019 = all_json['prospect_players']['draft']
draft_player_ids_2019 = []
draft_player_ranks_2019 = []
draft_player_first_name_2019 = []
draft_player_last_name_2019 = []
draft_player_position_2019 = []
for i in draft_prospect_list_2019:
    player_id_2019 = str(i['player_id'])
    draft_player_ids_2019.append(player_id_2019)                      
    rank_2019 = i["rank"]
    draft_player_ranks_2019.append(rank_2019)
    first_name_2019 = i["player_first_name"]
    draft_player_first_name_2019.append(first_name_2019)
    last_name_2019 = i["player_last_name"]
    draft_player_last_name_2019.append(last_name_2019)
    position_2019 = i["position"]
    draft_player_position_2019.append(position_2019)
    player_id_data_2019 = {'First Name' : draft_player_first_name_2019, 'Last Name' : draft_player_last_name_2019, 'Player ID' : draft_player_ids_2019, 'Position' : draft_player_position_2019, 'Player Rank' : draft_player_ranks_2019}

draft_player_id_dataframe_2019 = pd.DataFrame(player_id_data_2019)


###Part 3: Scraping every Top100 report into dataframes
pipeline_url = "http://m.mlb.com/gen/players/prospects/2019/"
draft_prospect_url_list_2019 = []
link_status_2019 = []
#making links
for i in draft_player_ids_2019:
    prospect_url = pipeline_url + i + '.json'
    draft_prospect_url_list_2019.append(prospect_url)

#testing links
for i in draft_prospect_url_list_2019:    
    response = requests.get(i)
    if response.status_code == 200:
        link = 'Link is working'
    else:
        link = 'Link does not work'
    link_status_2019.append(link)
    
#making initial list of player scout-reports/cards
draft_card_list_2019 = []
for i in draft_prospect_url_list_2019:    
    response = requests.get(i)
    player_card_framed_2019 = response.json()
    player_card_2019 = player_card_framed_2019['prospect_player']
    player_card_2019
    player_report_and_video_2019 = player_card_2019['content']
    player_report_2019 = player_report_and_video_2019['default']
    draft_card_list_2019.append(player_report_2019)



draft_report_trim_1_list_2019 = []
for i in draft_card_list_2019:
    trim = i.rfind('Overall: ') + 10
    report_trim_1 = i[trim:]
    draft_report_trim_1_list_2019.append(report_trim_1)

draft_report_cleaner_list_2019 = []
for i in draft_report_trim_1_list_2019:
    if i.find('</a>') == -1:
        start = i.find('n<p>') + 3
    else:
        start = i.find('</a>') + 8
    trim_a = i[start:]
    draft_report_cleaner_list_2019.append(trim_a)

draft_reports_2019 = []
for i in draft_report_cleaner_list_2019:
    without_p = i.replace('<p>', '')
    without_p2 = without_p.replace('p>', '')
    without_n = without_p2.replace('\n', '')
    lowered = without_n.lower()
    draft_reports_2019.append(lowered)

draft_player_id_dataframe_2019['Written Report'] = draft_reports_2019
draft_top_one_hundred_data_2019 = draft_player_id_dataframe_2019


full_draft_2019 = []

for i in draft_reports_2019:
    
    draft_text_2019 = TextBlob(i)
   
    draft_score_2019 = draft_text_2019.sentiment
    
    full_draft_2019.append(draft_score_2019)


draft_top_one_hundred_data_2019['Sentiment Score'] = full_draft_2019

sorted_draft_2019 = draft_top_one_hundred_data_2019.sort_values(by=['Sentiment Score'],ascending = False)
sorted_draft_2019 = sorted_draft_2019.drop(['Written Report', 'Player ID'], axis = 1)
sorted_draft_2019

,First Name,Last Name,Position,Player Rank,Sentiment Score
153,Jared,Southard,RHP,154,"(0.3707894736842105, 0.5028947368421052)"
68,Matthew,Thompson,RHP,69,"(0.36895833333333333, 0.4852083333333333)"
0,Adley,Rutschman,C,1,"(0.3509848484848485, 0.563409090909091)"
161,Ricky,DeVito,RHP,162,"(0.34225490196078434, 0.49401960784313725)"
191,Nick,Paciorek,RHP,192,"(0.324025974025974, 0.33636363636363636)"
114,Alex,McFarlane,RHP,115,"(0.3211764705882353, 0.3754621848739496)"
173,Michael,McAvene,RHP,174,"(0.31506410256410255, 0.30705128205128207)"
13,Zack,Thompson,LHP,14,"(0.3103030303030303, 0.45452380952380944)"
4,JJ,Bleday,OF,5,"(0.3024074074074074, 0.3838359788359788)"
154,Jake,Agnos,LHP,155,"(0.2948717948717949, 0.29566074950690335)"


In [5]:
#The 2018 MLB Draft Class

mlb_json_id_url = 'http://m.mlb.com/gen/players/prospects/2018/playerProspects.json'
response = requests.get(mlb_json_id_url)
response.status_code
all_json = response.json()
draft_prospect_list_2018 = all_json['prospect_players']['draft']
draft_player_ids_2018 = []
draft_player_ranks_2018 = []
draft_player_first_name_2018 = []
draft_player_last_name_2018 = []
draft_player_position_2018 = []
for i in draft_prospect_list_2018:
    player_id_2018 = str(i['player_id'])
    draft_player_ids_2018.append(player_id_2018)                      
    rank_2018 = i["rank"]
    draft_player_ranks_2018.append(rank_2018)
    first_name_2018 = i["player_first_name"]
    draft_player_first_name_2018.append(first_name_2018)
    last_name_2018 = i["player_last_name"]
    draft_player_last_name_2018.append(last_name_2018)
    position_2018 = i["position"]
    draft_player_position_2018.append(position_2018)
    player_id_data_2018 = {'First Name' : draft_player_first_name_2018, 'Last Name' : draft_player_last_name_2018, 'Player ID' : draft_player_ids_2018, 'Position' : draft_player_position_2018, 'Player Rank' : draft_player_ranks_2018}

draft_player_id_dataframe_2018 = pd.DataFrame(player_id_data_2018)


###Part 3: Scraping every Top100 report into dataframes
pipeline_url = "http://m.mlb.com/gen/players/prospects/2018/"
draft_prospect_url_list_2018 = []
link_status_2018 = []
#making links
for i in draft_player_ids_2018:
    prospect_url = pipeline_url + i + '.json'
    draft_prospect_url_list_2018.append(prospect_url)

#testing links
for i in draft_prospect_url_list_2018:    
    response = requests.get(i)
    if response.status_code == 200:
        link = 'Link is working'
    else:
        link = 'Link does not work'
    link_status_2018.append(link)
    
#making initial list of player scout-reports/cards
draft_card_list_2018 = []
for i in draft_prospect_url_list_2018:    
    response = requests.get(i)
    player_card_framed_2018 = response.json()
    player_card_2018 = player_card_framed_2018['prospect_player']
    player_card_2018
    player_report_and_video_2018 = player_card_2018['content']
    player_report_2018 = player_report_and_video_2018['default']
    draft_card_list_2018.append(player_report_2018)



draft_report_trim_1_list_2018 = []
for i in draft_card_list_2018:
    trim = i.rfind('Overall: ') + 10
    report_trim_1 = i[trim:]
    draft_report_trim_1_list_2018.append(report_trim_1)

draft_report_cleaner_list_2018 = []
for i in draft_report_trim_1_list_2018:
    if i.find('</a>') == -1:
        start = i.find('n<p>') + 3
    else:
        start = i.find('</a>') + 8
    trim_a = i[start:]
    draft_report_cleaner_list_2018.append(trim_a)

draft_reports_2018 = []
for i in draft_report_cleaner_list_2018:
    without_p = i.replace('<p>', '')
    without_p2 = without_p.replace('p>', '')
    without_n = without_p2.replace('\n', '')
    lowered = without_n.lower()
    draft_reports_2018.append(lowered)

draft_player_id_dataframe_2018['Written Report'] = draft_reports_2018
draft_top_one_hundred_data_2018 = draft_player_id_dataframe_2018


full_draft_2018 = []

for i in draft_reports_2018:
    
    draft_text_2018 = TextBlob(i)
   
    draft_score_2018 = draft_text_2018.sentiment
    
    full_draft_2018.append(draft_score_2018)


draft_top_one_hundred_data_2018['Sentiment Score'] = full_draft_2018

sorted_draft_2018 = draft_top_one_hundred_data_2018.sort_values(by=['Sentiment Score'],ascending = False)
sorted_draft_2018 = sorted_draft_2018.drop(['Written Report', 'Player ID'], axis = 1)
sorted_draft_2018

,First Name,Last Name,Position,Player Rank,Sentiment Score
139,Trey,Dillard,RHP,140,"(0.31869047619047625, 0.5707936507936507)"
140,Korey,Holland,OF,141,"(0.31529411764705884, 0.39563025210084035)"
105,Gage,Canning,OF,106,"(0.2994791666666667, 0.4020833333333333)"
24,Brice,Turang,SS,25,"(0.29708238851095997, 0.47519652305366594)"
43,Sean,Hjelle,RHP,44,"(0.29000000000000004, 0.5021052631578948)"
66,Lyon,Richardson,RHP,67,"(0.2804242424242424, 0.4856363636363637)"
34,Tristan,Beck,RHP,35,"(0.27785714285714286, 0.3729365079365079)"
150,Austin,Cox,LHP,151,"(0.2766666666666667, 0.4663461538461539)"
79,Braydon,Fisher,RHP,80,"(0.27175, 0.37575)"
65,Gunnar,Hoglund,RHP,66,"(0.2686746031746032, 0.4745396825396825)"


In [6]:
#The 2017 MLB Draft Class

mlb_json_id_url = 'http://m.mlb.com/gen/players/prospects/2017/playerProspects.json'
response = requests.get(mlb_json_id_url)
response.status_code
all_json = response.json()
draft_prospect_list_2017 = all_json['prospect_players']['draft']
draft_player_ids_2017 = []
draft_player_ranks_2017 = []
draft_player_first_name_2017 = []
draft_player_last_name_2017 = []
draft_player_position_2017 = []
for i in draft_prospect_list_2017:
    player_id_2017 = str(i['player_id'])
    draft_player_ids_2017.append(player_id_2017)                      
    rank_2017 = i["rank"]
    draft_player_ranks_2017.append(rank_2017)
    first_name_2017 = i["player_first_name"]
    draft_player_first_name_2017.append(first_name_2017)
    last_name_2017 = i["player_last_name"]
    draft_player_last_name_2017.append(last_name_2017)
    position_2017 = i["position"]
    draft_player_position_2017.append(position_2017)
    player_id_data_2017 = {'First Name' : draft_player_first_name_2017, 'Last Name' : draft_player_last_name_2017, 'Player ID' : draft_player_ids_2017, 'Position' : draft_player_position_2017, 'Player Rank' : draft_player_ranks_2017}

draft_player_id_dataframe_2017 = pd.DataFrame(player_id_data_2017)


###Part 3: Scraping every Top100 report into dataframes
pipeline_url = "http://m.mlb.com/gen/players/prospects/2017/"
draft_prospect_url_list_2017 = []
link_status_2017 = []
#making links
for i in draft_player_ids_2017:
    prospect_url = pipeline_url + i + '.json'
    draft_prospect_url_list_2017.append(prospect_url)

#testing links
for i in draft_prospect_url_list_2017:    
    response = requests.get(i)
    if response.status_code == 200:
        link = 'Link is working'
    else:
        link = 'Link does not work'
    link_status_2017.append(link)
    
#making initial list of player scout-reports/cards
draft_card_list_2017 = []
for i in draft_prospect_url_list_2017:    
    response = requests.get(i)
    player_card_framed_2017 = response.json()
    player_card_2017 = player_card_framed_2017['prospect_player']
    player_card_2017
    player_report_and_video_2017 = player_card_2017['content']
    player_report_2017 = player_report_and_video_2017['default']
    draft_card_list_2017.append(player_report_2017)



draft_report_trim_1_list_2017 = []
for i in draft_card_list_2017:
    trim = i.rfind('Overall: ') + 10
    report_trim_1 = i[trim:]
    draft_report_trim_1_list_2017.append(report_trim_1)

draft_report_cleaner_list_2017 = []
for i in draft_report_trim_1_list_2017:
    if i.find('</a>') == -1:
        start = i.find('n<p>') + 3
    else:
        start = i.find('</a>') + 8
    trim_a = i[start:]
    draft_report_cleaner_list_2017.append(trim_a)

draft_reports_2017 = []
for i in draft_report_cleaner_list_2017:
    without_p = i.replace('<p>', '')
    without_p2 = without_p.replace('p>', '')
    without_n = without_p2.replace('\n', '')
    lowered = without_n.lower()
    draft_reports_2017.append(lowered)

draft_player_id_dataframe_2017['Written Report'] = draft_reports_2017
draft_top_one_hundred_data_2017 = draft_player_id_dataframe_2017


full_draft_2017 = []

for i in draft_reports_2017:
    
    draft_text_2017 = TextBlob(i)
   
    draft_score_2017 = draft_text_2017.sentiment
    
    full_draft_2017.append(draft_score_2017)


draft_top_one_hundred_data_2017['Sentiment Score'] = full_draft_2017

sorted_draft_2017 = draft_top_one_hundred_data_2017.sort_values(by=['Sentiment Score'],ascending = False)
sorted_draft_2017 = sorted_draft_2017.drop(['Written Report', 'Player ID'], axis = 1)
sorted_draft_2017

,First Name,Last Name,Position,Player Rank,Sentiment Score
57,Blaine,Knight,RHP,58,"(0.4089546351084813, 0.48532544378698217)"
137,Jared,Oliva,OF,138,"(0.37954415954415954, 0.5772079772079772)"
151,David,Parkinson,LHP,152,"(0.35090909090909095, 0.35666666666666663)"
36,Jeter,Downs,SS,37,"(0.3454385964912281, 0.4850877192982456)"
177,Jayson,Rose,RHP,178,"(0.3368421052631579, 0.6342105263157894)"
71,MJ,Melendez,C,72,"(0.3276388888888889, 0.6441666666666669)"
192,Zach,Schellenger,RHP,193,"(0.3173076923076923, 0.46923076923076923)"
158,Brigham,Hill,RHP,159,"(0.3078571428571429, 0.3457142857142857)"
23,Seth,Romero,LHP,24,"(0.294875, 0.49455952380952384)"
110,Jackson,Rutledge,RHP,111,"(0.293032212885154, 0.43690476190476196)"


In [7]:
#The 2016 MLB Draft Class

mlb_json_id_url = 'http://m.mlb.com/gen/players/prospects/2016/playerProspects.json'
response = requests.get(mlb_json_id_url)
response.status_code
all_json = response.json()
draft_prospect_list_2016 = all_json['prospect_players']['draft']
draft_player_ids_2016 = []
draft_player_ranks_2016 = []
draft_player_first_name_2016 = []
draft_player_last_name_2016 = []
draft_player_position_2016 = []
for i in draft_prospect_list_2016:
    player_id_2016 = str(i['player_id'])
    draft_player_ids_2016.append(player_id_2016)                      
    rank_2016 = i["rank"]
    draft_player_ranks_2016.append(rank_2016)
    first_name_2016 = i["player_first_name"]
    draft_player_first_name_2016.append(first_name_2016)
    last_name_2016 = i["player_last_name"]
    draft_player_last_name_2016.append(last_name_2016)
    position_2016 = i["position"]
    draft_player_position_2016.append(position_2016)
    player_id_data_2016 = {'First Name' : draft_player_first_name_2016, 'Last Name' : draft_player_last_name_2016, 'Player ID' : draft_player_ids_2016, 'Position' : draft_player_position_2016, 'Player Rank' : draft_player_ranks_2016}

draft_player_id_dataframe_2016 = pd.DataFrame(player_id_data_2016)


###Part 3: Scraping every Top100 report into dataframes
pipeline_url = "http://m.mlb.com/gen/players/prospects/2016/"
draft_prospect_url_list_2016 = []
link_status_2016 = []
#making links
for i in draft_player_ids_2016:
    prospect_url = pipeline_url + i + '.json'
    draft_prospect_url_list_2016.append(prospect_url)

#testing links
for i in draft_prospect_url_list_2016:    
    response = requests.get(i)
    if response.status_code == 200:
        link = 'Link is working'
    else:
        link = 'Link does not work'
    link_status_2016.append(link)
    
#making initial list of player scout-reports/cards
draft_card_list_2016 = []
for i in draft_prospect_url_list_2016:    
    response = requests.get(i)
    player_card_framed_2016 = response.json()
    player_card_2016 = player_card_framed_2016['prospect_player']
    player_card_2016
    player_report_and_video_2016 = player_card_2016['content']
    player_report_2016 = player_report_and_video_2016['default']
    draft_card_list_2016.append(player_report_2016)



draft_report_trim_1_list_2016 = []
for i in draft_card_list_2016:
    trim = i.rfind('Overall: ') + 10
    report_trim_1 = i[trim:]
    draft_report_trim_1_list_2016.append(report_trim_1)

draft_report_cleaner_list_2016 = []
for i in draft_report_trim_1_list_2016:
    if i.find('</a>') == -1:
        start = i.find('n<p>') + 3
    else:
        start = i.find('</a>') + 8
    trim_a = i[start:]
    draft_report_cleaner_list_2016.append(trim_a)

draft_reports_2016 = []
for i in draft_report_cleaner_list_2016:
    without_p = i.replace('<p>', '')
    without_p2 = without_p.replace('p>', '')
    without_n = without_p2.replace('\n', '')
    lowered = without_n.lower()
    draft_reports_2016.append(lowered)

draft_player_id_dataframe_2016['Written Report'] = draft_reports_2016
draft_top_one_hundred_data_2016 = draft_player_id_dataframe_2016


full_draft_2016 = []

for i in draft_reports_2016:
    
    draft_text_2016 = TextBlob(i)
   
    draft_score_2016 = draft_text_2016.sentiment
    
    full_draft_2016.append(draft_score_2016)


draft_top_one_hundred_data_2016['Sentiment Score'] = full_draft_2016

sorted_draft_2016 = draft_top_one_hundred_data_2016.sort_values(by=['Sentiment Score'],ascending = False)
sorted_draft_2016 = sorted_draft_2016.drop(['Written Report', 'Player ID'], axis = 1)
sorted_draft_2016

,First Name,Last Name,Position,Player Rank,Sentiment Score
27,T.J.,Zeuch,RHP,28,"(0.3695138888888889, 0.5322222222222223)"
117,Erik,Miller,LHP,118,"(0.3266666666666667, 0.41507936507936505)"
104,Ulysses,Cantu,3B/C,105,"(0.3266666666666666, 0.47161375661375665)"
147,Drew,Harrington,LHP,148,"(0.32202380952380955, 0.3833333333333333)"
36,Kevin,Gowdy,RHP,37,"(0.3158823529411765, 0.5645098039215687)"
42,Anthony,Kay,LHP,43,"(0.30222222222222217, 0.4399999999999999)"
9,Braxton,Garrett,LHP,10,"(0.30029137529137534, 0.42738927738927723)"
56,Daulton,Jefferies,RHP,57,"(0.29060150375939847, 0.4476190476190477)"
164,J.C.,Flowers,RHP/OF,165,"(0.2807692307692307, 0.4717194570135745)"
127,Tyler,Mondile,RHP,128,"(0.28003819709702066, 0.5026203208556149)"


In [8]:
#The 2014 MLB Draft Class

mlb_json_id_url = 'http://m.mlb.com/gen/players/prospects/2014/playerProspects.json'
response = requests.get(mlb_json_id_url)
response.status_code
all_json = response.json()
draft_prospect_list_2014 = all_json['prospect_players']['draft']
draft_player_ids_2014 = []
draft_player_ranks_2014 = []
draft_player_first_name_2014 = []
draft_player_last_name_2014 = []
draft_player_position_2014 = []
for i in draft_prospect_list_2014:
    player_id_2014 = str(i['player_id'])
    draft_player_ids_2014.append(player_id_2014)                      
    rank_2014 = i["rank"]
    draft_player_ranks_2014.append(rank_2014)
    first_name_2014 = i["player_first_name"]
    draft_player_first_name_2014.append(first_name_2014)
    last_name_2014 = i["player_last_name"]
    draft_player_last_name_2014.append(last_name_2014)
    position_2014 = i["position"]
    draft_player_position_2014.append(position_2014)
    player_id_data_2014 = {'First Name' : draft_player_first_name_2014, 'Last Name' : draft_player_last_name_2014, 'Player ID' : draft_player_ids_2014, 'Position' : draft_player_position_2014, 'Player Rank' : draft_player_ranks_2014}

draft_player_id_dataframe_2014 = pd.DataFrame(player_id_data_2014)


###Part 3: Scraping every Top100 report into dataframes
pipeline_url = "http://m.mlb.com/gen/players/prospects/2014/"
draft_prospect_url_list_2014 = []
link_status_2014 = []
#making links
for i in draft_player_ids_2014:
    prospect_url = pipeline_url + i + '.json'
    draft_prospect_url_list_2014.append(prospect_url)

#testing links
for i in draft_prospect_url_list_2014:    
    response = requests.get(i)
    if response.status_code == 200:
        link = 'Link is working'
    else:
        link = 'Link does not work'
    link_status_2014.append(link)
    
#making initial list of player scout-reports/cards
draft_card_list_2014 = []
for i in draft_prospect_url_list_2014:    
    response = requests.get(i)
    player_card_framed_2014 = response.json()
    player_card_2014 = player_card_framed_2014['prospect_player']
    player_card_2014
    player_report_and_video_2014 = player_card_2014['content']
    player_report_2014 = player_report_and_video_2014['default']
    draft_card_list_2014.append(player_report_2014)



draft_report_trim_1_list_2014 = []
for i in draft_card_list_2014:
    trim = i.rfind('Overall: ') + 10
    report_trim_1 = i[trim:]
    draft_report_trim_1_list_2014.append(report_trim_1)

draft_report_cleaner_list_2014 = []
for i in draft_report_trim_1_list_2014:
    if i.find('</a>') == -1:
        start = i.find('n<p>') + 3
    else:
        start = i.find('</a>') + 8
    trim_a = i[start:]
    draft_report_cleaner_list_2014.append(trim_a)

draft_reports_2014 = []
for i in draft_report_cleaner_list_2014:
    without_p = i.replace('<p>', '')
    without_p2 = without_p.replace('p>', '')
    without_n = without_p2.replace('\n', '')
    lowered = without_n.lower()
    draft_reports_2014.append(lowered)

draft_player_id_dataframe_2014['Written Report'] = draft_reports_2014
draft_top_one_hundred_data_2014 = draft_player_id_dataframe_2014


full_draft_2014 = []

for i in draft_reports_2014:
    
    draft_text_2014 = TextBlob(i)
   
    draft_score_2014 = draft_text_2014.sentiment
    
    full_draft_2014.append(draft_score_2014)


draft_top_one_hundred_data_2014['Sentiment Score'] = full_draft_2014

sorted_draft_2014 = draft_top_one_hundred_data_2014.sort_values(by=['Sentiment Score'],ascending = False)
sorted_draft_2014 = sorted_draft_2014.drop(['Written Report', 'Player ID'], axis = 1)
sorted_draft_2014

,First Name,Last Name,Position,Player Rank,Sentiment Score
133,Griffin,Canning,RHP,134,"(0.4143849206349206, 0.5973710317460317)"
198,Trevor,Oaks,RHP,199,"(0.4050000000000001, 0.575)"
23,Luis,Ortiz,RHP,24,"(0.37933333333333336, 0.4573333333333333)"
27,Foster,Griffin,LHP,28,"(0.3403030303030303, 0.546060606060606)"
44,Mike,Papi,1B/OF,45,"(0.328452380952381, 0.45678571428571424)"
199,Jordan,Yamomoto,RHP,200,"(0.32583829365079364, 0.4429365079365079)"
139,Dean,Deetz,RHP,140,"(0.3233333333333333, 0.4888461538461538)"
179,Austin,Robichaux,RHP,180,"(0.3204545454545455, 0.39790209790209785)"
77,Aramis,Garcia,C,78,"(0.314, 0.356)"
1,Carlos,Rodon,LHP,2,"(0.31323529411764706, 0.44411764705882356)"


In [9]:
#The 2013 MLB Draft Class

mlb_json_id_url = 'http://m.mlb.com/gen/players/prospects/2013/playerProspects.json'
response = requests.get(mlb_json_id_url)
response.status_code
all_json = response.json()
draft_prospect_list_2013 = all_json['prospect_players']['draft']
draft_player_ids_2013 = []
draft_player_ranks_2013 = []
draft_player_first_name_2013 = []
draft_player_last_name_2013 = []
draft_player_position_2013 = []
for i in draft_prospect_list_2013:
    player_id_2013 = str(i['player_id'])
    draft_player_ids_2013.append(player_id_2013)                      
    rank_2013 = i["rank"]
    draft_player_ranks_2013.append(rank_2013)
    first_name_2013 = i["player_first_name"]
    draft_player_first_name_2013.append(first_name_2013)
    last_name_2013 = i["player_last_name"]
    draft_player_last_name_2013.append(last_name_2013)
    position_2013 = i["position"]
    draft_player_position_2013.append(position_2013)
    player_id_data_2013 = {'First Name' : draft_player_first_name_2013, 'Last Name' : draft_player_last_name_2013, 'Player ID' : draft_player_ids_2013, 'Position' : draft_player_position_2013, 'Player Rank' : draft_player_ranks_2013}

draft_player_id_dataframe_2013 = pd.DataFrame(player_id_data_2013)


###Part 3: Scraping every Top100 report into dataframes
pipeline_url = "http://m.mlb.com/gen/players/prospects/2013/"
draft_prospect_url_list_2013 = []
link_status_2013 = []
#making links
for i in draft_player_ids_2013:
    prospect_url = pipeline_url + i + '.json'
    draft_prospect_url_list_2013.append(prospect_url)

#testing links
for i in draft_prospect_url_list_2013:    
    response = requests.get(i)
    if response.status_code == 200:
        link = 'Link is working'
    else:
        link = 'Link does not work'
    link_status_2013.append(link)
    
#making initial list of player scout-reports/cards
draft_card_list_2013 = []
for i in draft_prospect_url_list_2013:    
    response = requests.get(i)
    player_card_framed_2013 = response.json()
    player_card_2013 = player_card_framed_2013['prospect_player']
    player_card_2013
    player_report_and_video_2013 = player_card_2013['content']
    player_report_2013 = player_report_and_video_2013['default']
    draft_card_list_2013.append(player_report_2013)



draft_report_trim_1_list_2013 = []
for i in draft_card_list_2013:
    trim = i.rfind('Overall: ') + 10
    report_trim_1 = i[trim:]
    draft_report_trim_1_list_2013.append(report_trim_1)

draft_report_cleaner_list_2013 = []
for i in draft_report_trim_1_list_2013:
    if i.find('</a>') == -1:
        start = i.find('n<p>') + 3
    else:
        start = i.find('</a>') + 8
    trim_a = i[start:]
    draft_report_cleaner_list_2013.append(trim_a)

draft_reports_2013 = []
for i in draft_report_cleaner_list_2013:
    without_p = i.replace('<p>', '')
    without_p2 = without_p.replace('p>', '')
    without_n = without_p2.replace('\n', '')
    lowered = without_n.lower()
    draft_reports_2013.append(lowered)

draft_player_id_dataframe_2013['Written Report'] = draft_reports_2013
draft_top_one_hundred_data_2013 = draft_player_id_dataframe_2013


full_draft_2013 = []

for i in draft_reports_2013:
    
    draft_text_2013 = TextBlob(i)
   
    draft_score_2013 = draft_text_2013.sentiment
    
    full_draft_2013.append(draft_score_2013)


draft_top_one_hundred_data_2013['Sentiment Score'] = full_draft_2013

sorted_draft_2013 = draft_top_one_hundred_data_2013.sort_values(by=['Sentiment Score'],ascending = False)
sorted_draft_2013 = sorted_draft_2013.drop(['Written Report', 'Player ID'], axis = 1)
sorted_draft_2013

,First Name,Last Name,Position,Player Rank,Sentiment Score
12,Sean,Manaea,LHP,13,"(0.45803030303030307, 0.6333333333333333)"
58,Dustin,Driver,"RHP, 3B",59,"(0.39630559916274205, 0.5111132914704343)"
29,Bobby,Wahl,RHP,30,"(0.37179487179487186, 0.5653846153846154)"
13,Marco,Gonzales,LHP,14,"(0.37139682539682545, 0.5647619047619048)"
78,Colby,Suggs,RHP,79,"(0.3416666666666667, 0.6333333333333333)"
10,DJ,Peterson,1B/3B,11,"(0.3156655844155844, 0.5341382575757576)"
32,Kevin,Ziomek,LHP,33,"(0.30666666666666664, 0.5529166666666666)"
97,Stephen,Gonsalves,LHP/OF,98,"(0.30166666666666664, 0.5394444444444445)"
20,Phillip,Ervin,OF,21,"(0.3008412698412698, 0.5811111111111112)"
4,Austin,Meadows,OF/1B,5,"(0.3007852564102564, 0.5766586538461538)"
